In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
Image_size = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/plant-village/PlantVillage",
    shuffle=True,
    image_size=(Image_size, Image_size),
    batch_size = BATCH_SIZE
)

In [ ]:
class_names = dataset.class_names

In [ ]:
n_classes = len(class_names)

In [ ]:
plt.figure(figsize=(18,18))
for image_batch, label_batch in dataset.take(1):
  for i in range(12):
    ax = plt.subplot(3,4,i+1)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    # print(image_batch[0].numpy())
    plt.title(class_names[label_batch[i]])
    plt.axis(False)

Divide Data into train validation and test
80%,10% & 10% respectively

In [ ]:
train_set = dataset.take(54)
len(train_set)

In [ ]:
rem = dataset.skip(54)
len(rem)

In [ ]:
len(dataset) * .1


In [ ]:
val_set = rem.take(6)
test_set = rem.skip(6)

In [ ]:
len(test_set)

In [ ]:
def get_train_val_test_split(ds, train_ratio=0.8, test_ratio=0.1, val_ratio=0.1, shuffle=True, shuffle_size= 10000):
  ds_size = len(ds)
  if(shuffle):
    ds = ds.shuffle(shuffle_size, seed=12)
  train_size = int(train_ratio*ds_size)
  test_size = int(test_ratio * ds_size)
  val_size = int(val_ratio * ds_size)
  train_ds = ds.take(train_size)
  val_ds = ds.skip(train_size).take(val_size)
  test_ds = ds.skip(train_size).skip(val_size)
  return train_ds, test_ds, val_ds

In [ ]:
train_ds, test_ds, val_ds = get_train_val_test_split(dataset)

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
resize_rescaling = tf.keras.Sequential([
    keras.layers.experimental.preprocessing.Resizing(Image_size,Image_size),
    keras.layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augumentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
input_shape = (BATCH_SIZE, Image_size, Image_size, CHANNELS)

model = keras.models.Sequential([
    resize_rescaling,
    data_augumentation,
    keras.layers.Conv2D(32,(3,3),activation="relu", input_shape = input_shape),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(128,(3,3),activation="relu"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(128,(3,3),activation="relu"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64,(3,3),activation="relu"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32,(3,3),activation="relu"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dense(n_classes, activation="softmax")
])

model.build(input_shape = input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

In [ ]:
model.fit(
    train_ds,
    epochs=EPOCHS,batch_size = BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)